In [1]:
import pandas as pd
import sklearn
# from sklearn import cross_validation
# from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
dataset = pd.read_excel(r'./200416_label0_dataset_sentence.xlsx')

In [2]:
tmp = []
for i in dataset['sender']:
    if i == 'customer':
        tmp.append(0)
    else:
        tmp.append(1)
dataset['sender_num'] = tmp
dataset.head()

,id,round,sender,texts,max_label,round_label,labels,sender_num
0,4227729258237820,1,customer,内涵 段子 联通 皮 点赞 中国联通 中国联通 客服 掌上 营业厅 内涵 段子 话题 封 郑...,3,2,1,0
1,4227729258237820,2,helpdesk,u,6,4,0,1
2,4227729258237820,3,customer,夸夸,3,0,1,0
3,4227729258237820,4,helpdesk,*,6,4,0,1
4,4121001149457180,1,customer,距离 反映 问题 已经 一个 星期 花粉 助手 D 荣耀 honor 荣耀 手机 华为 终端...,2,2,1,0


In [3]:
texts = list(dataset["texts"])
max_label = list(dataset["max_label"])
Round = list(dataset["round"])
sender_num = list(dataset["sender_num"])

In [4]:
import os
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.decomposition import TruncatedSVD

In [5]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

# train / dev

In [6]:
def baseline_model(texts,max_label,sender_num,Round):
    texts = [str (item) for item in texts]
    x_train1 = tfidf_vectorizer.fit_transform(texts).toarray()
#     x_test = tfidf_vectorizer.transform(x_test).toarray()
    svd = TruncatedSVD(n_components=400).fit(x_train1)
    reduced = svd.transform(x_train1)
    total = reduced.tolist()
    for item in range(len(total)):
        total[item] = np.append(total[item], sender_num[item])
    for item in range(len(total)):
        total[item] = np.append(total[item], Round[item])
        
    total = np.array(total)
    max_label = np.array(max_label)    
        
    x_train=total[1755:]
    x_test=total[0:1755]
    y_train=max_label[1755:]
    y_test=max_label[0:1755]    
    
    Models = ['Bernoulli NB','Svm (linear)','Logistic Regression',
              'Random Forest','kNN','Decision Tree']
    function = [BernoulliNB(),svm.SVC(kernel="linear"),LogisticRegression(),
              RandomForestClassifier(),KNeighborsClassifier(),DecisionTreeClassifier()]
    perform_f1 = []
    perform_acc = []
    
    
    for i in tqdm(range(len(function))):
        model = function[i]
#         performance = cross_val_score(model, tfidf_vectorizer.fit_transform(texts), labels, cv=10, scoring'accuracy')
        
        func = str(function[i])
        print("==== ", func[0:func.index('(')], " ====")

        model.fit(x_train, y_train)
        model.score(x_test, y_test)
        e = y_test
        p = model.predict(x_test)
        print(metrics.classification_report(e,p))
        perform_f1.append(metrics.f1_score(e,p,average='macro'))
        perform_acc.append(metrics.accuracy_score(e,p))
        
    result_f1_table = pd.DataFrame({"Models":Models,"Result f1 scores":perform_f1})
    result_acc_table = pd.DataFrame({"Models":Models,"Result acc scores":perform_acc})
    return result_f1_table, result_acc_table

In [7]:
import numpy as np
score = baseline_model(texts,max_label,sender_num,Round)

====  BernoulliNB  ====
              precision    recall  f1-score   support

           0       0.68      0.77      0.72       477
           1       0.30      0.43      0.36        23
           2       0.71      0.69      0.70       374
           3       0.11      0.01      0.02       101
           4       0.88      0.91      0.89       662
           5       0.15      0.30      0.20        33
           6       0.38      0.29      0.33        85

    accuracy                           0.72      1755
   macro avg       0.46      0.49      0.46      1755
weighted avg       0.70      0.72      0.71      1755

====  SVC  ====


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.82      0.99      0.89       477
           1       0.50      0.04      0.08        23
           2       0.95      0.99      0.97       374
           3       0.20      0.01      0.02       101
           4       0.87      1.00      0.93       662
           5       0.00      0.00      0.00        33
           6       0.94      0.20      0.33        85

    accuracy                           0.87      1755
   macro avg       0.61      0.46      0.46      1755
weighted avg       0.82      0.87      0.82      1755

====  LogisticRegression  ====


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

           0       0.83      0.97      0.89       477
           1       0.33      0.26      0.29        23
           2       0.95      0.99      0.97       374
           3       0.40      0.04      0.07       101
           4       0.88      0.99      0.93       662
           5       0.25      0.03      0.05        33
           6       0.80      0.28      0.42        85

    accuracy                           0.87      1755
   macro avg       0.63      0.51      0.52      1755
weighted avg       0.83      0.87      0.83      1755

====  RandomForestClassifier  ====
              precision    recall  f1-score   support

           0       0.83      0.97      0.89       477
           1       0.29      0.09      0.13        23
           2       0.95      0.99      0.97       374
           3       0.33      0.05      0.09       101
           4       0.88      0.99      0.94       662
           5       0.50      0.03      0.06

# 10fold

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
def baseline_model(texts,labels,sender_num,Round):
    texts = [str (item) for item in texts]
    x_train = tfidf_vectorizer.fit_transform(texts).toarray()
#     x_test = tfidf_vectorizer.transform(x_test)
    svd = TruncatedSVD(n_components=400).fit(x_train)
    reduced = svd.transform(x_train)
    total = reduced.tolist()
    for item in range(len(total)):
        total[item] = np.append(total[item], sender_num[item])
    for item in range(len(total)):
        total[item] = np.append(total[item], Round[item])
        
    total = np.array(total)
    labels = np.array(labels)        
        
    
    Models = ['Bernoulli NB','Svm (linear)','Logistic Regression',
              'Random Forest','kNN','Decision Tree']
    function = [BernoulliNB(),svm.SVC(kernel="linear"),LogisticRegression(),
              RandomForestClassifier(),KNeighborsClassifier(),DecisionTreeClassifier()]
    perform_f1 = []
    perform_acc = []
    perform_recall = []
    perform_precision = []
    
    for i in tqdm(range(len(function))):
        model = function[i]
        performance = cross_val_score(model, total, labels, cv=10, scoring='accuracy')
        performance1 = cross_val_score(model, total, labels, cv=10, scoring='f1_weighted')
        performance2 = cross_val_score(model, total, labels, cv=10, scoring='recall')
        performance3 = cross_val_score(model, total, labels, cv=10, scoring='precision')
        
        func = str(function[i])
        print("==== ", func[0:func.index('(')], " ====")
        perform_acc.append(performance.mean())
        perform_f1.append(performance1.mean())
        perform_recall.append(performance2.mean())
        perform_precision.append(performance3.mean())

#         model.fit(x_train, y_train)
#         model.score(x_test, y_test)
#         e = y_test
#         p = model.predict(x_test)
#         print(metrics.classification_report(e,p))
#         perform_f1.append(metrics.f1_score(e,p,average='macro'))
#         perform_acc.append(metrics.accuracy_score(e,p))
        
    result_f1_table = pd.DataFrame({"Models":Models,"f1-scores":perform_f1,"accuracy":perform_acc
                                   ,"recall":perform_recall,"precision":perform_precision})
#     result_acc_table = pd.DataFrame({"Models":Models,"Result acc scores":perform_acc})
    return result_f1_table

In [ ]:
import numpy as np
score1 = baseline_model(texts,labels,sender_num,Round)

In [ ]:
score1